### DecisionTreeClassifier

In [6]:
import numpy as np
from numpy import linalg
import _pickle as cPickle
from collections import defaultdict
import sys, re
import pandas as pd
import codecs

### Vocabulary dictionary : word_idx_map

In [35]:
def build_data_cv(data_folder, cv=10):
    """
    Loads data and split into 10 folds.
    """
    revs = []
    label_0_file, label_1_file, label_2_file = data_folder #["__label__0.txt","__label__1.txt", "__label__2.txt"]    
    word_idx_map = dict()
    word_idx = 1
    for label, file in enumerate(data_folder): 
        with codecs.open(file, "rb", "utf-8", "ignore") as f:
            for line in f:       
                rev = []
                rev.append(line.strip())
                orig_rev = " ".join(rev).lower()

                words = set(orig_rev.split())
                for word in words:
                    if word not in word_idx_map:
                        word_idx_map[word] = word_idx
                        word_idx += 1

                datum  = {"y": label, 
                          "text": orig_rev,                             
                          "num_words": len(orig_rev.split()),
                          "split": np.random.randint(0, cv)}
                revs.append(datum)
    return revs, word_idx_map

In [36]:
data_folder = ["__label__0.txt","__label__1.txt", "__label__2.txt"]    

print("loading data...")      
revs, word_idx_map = build_data_cv(data_folder, cv=10)
print("Data loaded")

loading data...
Data loaded


### Dataset content
Get data from twitter (reference: http://tech.wonderpla.net/entry/2017/10/10/110000)
- Label 0
    - KEYWORD = "芸能 OR アニメ OR 漫画 OR ドラマ OR ゲーム"            #エンタメ系のキーワード
    - CLASS_LABEL = "\__label__0"

- Label 1
    - KEYWORD = "美容 OR サロン OR エステ OR 化粧 OR 保湿"            #美容系のキーワード
    - CLASS_LABEL = "\__label__1"

- Label 2
    - KEYWORD = "日常 OR 料理 OR 家事 OR 収納 OR 家具"            #暮らし系のキーワード
    - CLASS_LABEL = "\__label__2"

In [103]:
label_names = ['エンタメ系', '美容系', '暮らし系']

#### See data sample

In [12]:
df = pd.DataFrame(revs)

# label 0: Entertainment 
df[df['y'] == 0].sample(2)

num_words  split                                               text  y
108         60      7  # ピザ ハット 39 ルーレット に ご 応募 ありがとう ござい ます !(* ⁰ ▿ ...  0
260         60      4  # ピザ ハット 39 ルーレット に ご 応募 ありがとう ござい ます !(* ⁰ ▿ ...  0

In [13]:
# label 1 : Beauty
df[df['y'] == 1].sample(2)

num_words  split                                               text  y
909          31      5  普段 濃い 化粧 全然 し ない から 無駄 に 自 撮り し て た やつ な dc お ...  1
1159         39      0  今日 は 幼稚園 の ママ の ランチ 会 な ん だ けど 、 化粧 す べき か どうか...  1

In [14]:
# label 2: Life
df[df['y'] == 2].sample(2)

num_words  split                                               text  y
1464         28      1  車 じゃ ない と なかなか 行け ない の です が 、 地元 の 海鮮 と タイ 料理 ...  2
1835         55      8  《 8 / 30 newopen 》 ドリンク 総数 約 １ ０ ０ 種類 ♪ ワイン や ...  2

In [21]:
vocab_size = len(word_idx_map)
print('revs',len(revs)) # number of sentence
print('word_idx_map', vocab_size)

revs 2068
word_idx_map 10329


In [17]:
revs[1]

{'y': 0,
 'text': 'え 、 サラリーマン し ながら 、 商業 漫画 の 仕事 を し て 、 ツイッター に 定期 的 に 漫画 を あげ て 、 さらに コミケ の 原稿 を 作る ！ ？',
 'num_words': 32,
 'split': 6}

### Make BOW vectors
make each sentence an BOW vector using word_idx_map

In [55]:
def make_bow_from_sent(revs, word_idx_map, cv, vocab_size):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, test = [], []
    for rev in revs:
        sent = rev["text"]
        bow_vec = np.zeros(vocab_size + 1)
        bow_vec[0] = rev['y'] # Add true label at the beginning
        words = sent.split()
        for word in words:
            bow_vec[word_idx_map[word]] += 1

        if rev["split"]== cv:  # "split" is random number of np.random.randint(0,10)
            test.append(bow_vec)        
        else:  
            train.append(bow_vec)   
    train = np.array(train, dtype="int")
    test = np.array(test, dtype="int")
    return [train, test] 

In [56]:
cv = 0
datasets = make_bow_from_sent(revs, word_idx_map, cv, vocab_size)

#### Example of sentence to BOW

In [60]:
revs[0]

{'y': 0,
 'text': 'かっこいい 、 わくわく する 漫画 も 気 に なっ てる けど なかなか 読め て ない ん だ よ ねぇ 、 満喫 行こ う か な と 思っ て も 家 で ゴロゴロ っと し ちゃう',
 'num_words': 35,
 'split': 2}

In [64]:
datasets[0][0]

array([0, 1, 1, ..., 0, 0, 0])

### Implement DecisionTreeClassifier

In [119]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state = 0)

In [120]:
train_image = datasets[0][:, 1:]
train_label = datasets[0][:, 0]
test_image = datasets[1][:, 1:]
test_label = datasets[1][:, 0]

In [121]:
print('train data size:', train_image.shape)
print('train data size:', train_label.shape)
print('test data size:', test_image.shape)
print('train data size:', test_label.shape)

train data size: (1857, 10329)
train data size: (1857,)
test data size: (211, 10329)
train data size: (211,)


In [122]:
clf.fit(train_image, train_label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [123]:
print('Accuracy:', clf.score(test_image, test_label))

Accuracy: 0.990521327014


In [124]:
print('Accuracy:', clf.score(train_image, train_label))

Accuracy: 0.996768982229


### Predict single inputted text

#### Input the text to predict (change the text below)

In [75]:
# text_to_predict = "お風呂掃除でいつも落ちなかった溝にある黒カビが家事えもんの塩素系漂白剤＋片栗粉でほぼ真っ白になって感動 家事えもんのテクニック凄い～！"

text_to_predict = "コスメの最安値が見つけられるアプリ💄💋メイク動画とか 美容情報も載ってるし最高😆🙌📲http://goo.gl/K5Fmea 女子にはほんとに助かる〜💗"

In [91]:
from process_data import process_single_txt

In [96]:
def make_bow_from_single_sent(sent, word_idx_map, vocab_size):
    """
    Transforms sentences into a 2-d matrix.
    """
    test = []
    bow_vec = np.zeros(vocab_size + 1)
    bow_vec[0] = 0 # Add dummy true label at the beginning
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            bow_vec[word_idx_map[word]] += 1

    test.append(bow_vec)
    test = np.array(test, dtype="int")
    return test


In [93]:
processed_sent = process_single_txt(text_to_predict)
single_bow = make_bow_from_single_sent(processed_sent, word_idx_map, vocab_size)

1行処理済み


In [108]:
predected = clf.predict(single_bow[:, 1:]).item()
print('Predicted:', predected, label_names[predected])

Predicted: 1 美容系
